In [2]:
!pip install --upgrade implicit

     |████████████████████████████████| 1.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 6.1 MB/s  eta 0:00:01
  Created wheel for implicit: filename=implicit-0.4.2-cp38-cp38-linux_x86_64.whl size=4883578 sha256=e1b2e7d24e1810999531902bf5080788a3e5fa065c773a4f92ec781ce212ab1a
  Stored in directory: /home/voki/.cache/pip/wheels/97/dd/5f/df702090a221c1b1cc4683950b6d086eeee98d37a547f20f8f
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [5]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [41]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [42]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id']
    weights = items_weights['weight']
    
    # передаем дополнительно веса
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [53]:
# подготовим данные по объему продаж
popular = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)

# посолим и прологарифмируем
popular['weight'] = popular['sales_value'].apply(lambda x: np.log(x+1e-5))
# приведем мин в 0 (сделаем неотрицательными)
popular['weight'] = popular['weight'] - popular['weight'].min()  

log_sum = popular['weight'].sum()
# найдем веса
popular['weight'] = popular['weight']/log_sum

weighted_popular = popular[['item_id', 'weight']]
print(weighted_popular.head())
weighted_popular['weight'].sum()


       item_id    weight
55470  6534178  0.000020
55430  6533889  0.000018
28895  1029743  0.000018
55465  6534166  0.000018
34707  1082185  0.000018


1.0

In [54]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular, n=5))
result.head(10)


CPU times: user 4.42 s, sys: 0 ns, total: 4.42 s
Wall time: 4.42 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[928887, 922092, 1978006, 5585331, 1011427]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[251023, 1053107, 883186, 506756, 6602558]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[870714, 1031549, 12949665, 9372920, 997479]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[3031923, 945246, 919948, 17209759, 10456180]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[9884271, 15716560, 10356819, 15485728, 106963]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[8199765, 1084310, 855335, 13215259, 1094229]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1470762, 845896, 996851, 8155556, 2617092]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[14025138, 1075178, 958114, 1437522, 1157947]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[12256364, 13988878, 13945142, 1096727, 8019644]"
9,16,"[1062973, 1082185, 13007710]","[6633038, 858391, 845725, 1139080, 10312178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [55]:
result = pd.read_csv('./data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [ ]:
# your_code

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code